# 💼 PyKwant Tutorial: Portfolio Management
This notebook demonstrates `pykwant.portfolio`, the module for managing collections of financial instruments.

In line with our functional philosophy, a **Portfolio** is simply a `list` of `Position` objects. We process these lists using higher-order functions (like `map` and `reduce`) to aggregate value and risk.

## 1. Setup and Imports

In [1]:
from datetime import date
from pykwant import portfolio, instruments, rates, dates

# Helper for clean output
def print_risk_report(report: dict[str, float]):
    print("-" * 40)
    for k, v in report.items():
        print(f"{k.capitalize():<25}: {v:>12.4f}")
    print("-" * 40)

## 2. Defining the Market
We need a Yield Curve to value our positions. Let's create a simple flat curve at 4%.

In [2]:
ref_date = date(2025, 1, 1)

def flat_curve(d: date) -> float:
    t = dates.act_365(ref_date, d)
    # Flat 4% continuous rate
    return rates.compound_factor(-0.04, t, frequency=0)

## 3. Creating Instruments
Let's imagine a portfolio with two distinct bonds:

1. **Short-Term Bond** (2 Years, 3% Coupon)

2. **Long-Term Bond** (10 Years, 5% Coupon)

In [3]:
bond_2y = instruments.FixedRateBond(
    face_value=instruments.Money(100.0),
    coupon_rate=0.03,
    start_date=ref_date,
    maturity_date=date(2027, 1, 1),
    frequency_months=12,
    day_count=dates.thirty_360
)

bond_10y = instruments.FixedRateBond(
    face_value=instruments.Money(100.0),
    coupon_rate=0.05,
    start_date=ref_date,
    maturity_date=date(2035, 1, 1),
    frequency_months=12,
    day_count=dates.thirty_360
)

## 4. Building the Portfolio
A Position associates an instrument with a quantity (positive for Long, negative for Short).

We are **Long** 1000 units of the 2Y Bond and **Short** 500 units of the 10Y Bond.

In [4]:
# Create Positions
pos_long = portfolio.Position(instrument=bond_2y, quantity=1000.0)
pos_short = portfolio.Position(instrument=bond_10y, quantity=-500.0)

# Create Portfolio (List of Positions)
my_portfolio = [pos_long, pos_short]

print(f"Portfolio created with {len(my_portfolio)} positions.")

Portfolio created with 2 positions.


## 5. Portfolio Valuation (NPV)
The `portfolio_npv` function sums the present value of all positions.

In [5]:
total_value = portfolio.portfolio_npv(my_portfolio, flat_curve, ref_date)

print(f"Total Portfolio NPV: {total_value:,.2f}")

Total Portfolio NPV: 44,261.09


## 6. Risk Aggregation
The `portfolio_risk` function calculates the aggregated risk metrics.

* **Total Market Value**: Net Liquidation Value.

* **Total DV01**: Net sensitivity to a 1bp parallel shift.

* **Portfolio Duration**: The duration of the aggregate portfolio (weighted average).

In this case, since we are Long (low duration) vs Short (high duration), we expect a significant hedging effect.

In [6]:
risk_report = portfolio.portfolio_risk(my_portfolio, flat_curve, ref_date)

print("\n--- Portfolio Risk Report ---")
print_risk_report(risk_report)

# Interpretation:
# If Total DV01 is close to 0, the portfolio is "Duration Neutral" or hedged against small parallel shifts.


--- Portfolio Risk Report ---
----------------------------------------
Total_market_value       :   44261.0876
Total_dv01               :     -24.6692
Portfolio_duration       :      -5.5736
Positions_count          :       2.0000
----------------------------------------


## 7. Exposure Analysis
Sometimes we want to see risk grouped by maturity buckets. The `exposure_by_maturity_year` function groups positions by their maturity year and sums the market value.

In [7]:
exposure = portfolio.exposure_by_maturity_year(my_portfolio, flat_curve, ref_date)

print("\n--- Exposure by Maturity ---")
for year, value in sorted(exposure.items()):
    print(f"Year {year}: {value:>12,.2f}")


--- Exposure by Maturity ---
Year 2027:    97,963.35
Year 2035:   -53,702.26
